In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load data
pc3 = pd.read_csv(r'/content/pc3_csv.csv')
print('pc3 shape: ', pc3.shape)
pc3.head()

pc3 shape:  (1563, 38)


,LOC_BLANK,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,...,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL,LABEL
0,2,1,0,0,0,0,1,0.10,0,0.00,...,2,0.08,28,29,9,5,13,0.00,10,False
1,1,1,4,0,0,0,1,0.07,0,0.00,...,6,0.06,52,55,26,14,16,0.00,14,False
2,27,19,1,4,13,26,11,0.26,12,2.17,...,25,0.13,58,78,30,24,83,30.91,42,False
3,2,17,2,0,0,24,9,0.47,8,3.00,...,25,0.41,73,81,23,20,22,0.00,19,False
4,6,1,1,0,2,0,1,0.11,0,0.00,...,3,0.06,19,23,15,7,18,18.18,9,False


In [3]:
pc3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1563 entries, 0 to 1562
Data columns (total 38 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   LOC_BLANK                        1563 non-null   int64  
 1   BRANCH_COUNT                     1563 non-null   int64  
 2   CALL_PAIRS                       1563 non-null   int64  
 3   LOC_CODE_AND_COMMENT             1563 non-null   int64  
 4   LOC_COMMENTS                     1563 non-null   int64  
 5   CONDITION_COUNT                  1563 non-null   int64  
 6   CYCLOMATIC_COMPLEXITY            1563 non-null   int64  
 7   CYCLOMATIC_DENSITY               1563 non-null   float64
 8   DECISION_COUNT                   1563 non-null   int64  
 9   DECISION_DENSITY                 1563 non-null   float64
 10  DESIGN_COMPLEXITY                1563 non-null   int64  
 11  DESIGN_DENSITY                   1563 non-null   float64
 12  EDGE_COUNT          

In [4]:
target = 'LABEL'
y = pc3[target]
x=pc3.drop(['LABEL'], axis=1)
x.shape

(1563, 37)

In [5]:
def mad(array, etc): #etc is dummy parameter
  fmad = np.sum(np.abs(array -np.mean(array, axis=0 )), axis=0)/array.shape[0]
  return fmad

In [6]:
mad(x, 0)

LOC_BLANK                              6.336536
BRANCH_COUNT                           8.783237
CALL_PAIRS                             2.028846
LOC_CODE_AND_COMMENT                   1.783156
LOC_COMMENTS                           5.877647
CONDITION_COUNT                       15.435310
CYCLOMATIC_COMPLEXITY                  4.610937
CYCLOMATIC_DENSITY                     0.136905
DECISION_COUNT                         7.214248
DECISION_DENSITY                       0.877894
DESIGN_COMPLEXITY                      2.241582
DESIGN_DENSITY                         0.270502
EDGE_COUNT                            19.092232
ESSENTIAL_COMPLEXITY                   2.156869
ESSENTIAL_DENSITY                      0.264112
LOC_EXECUTABLE                        17.966083
PARAMETER_COUNT                        0.966179
HALSTEAD_CONTENT                      25.885133
HALSTEAD_DIFFICULTY                    9.577824
HALSTEAD_EFFORT                    50310.128874
HALSTEAD_ERROR_EST                     0

In [7]:
from sklearn.feature_selection import SelectKBest # this will get the features with highest MAD value and higher the MAD, higher the discriminatory power.
selector = SelectKBest(mad, k=5) # set k value according to need. it will get top k number of features that will have high discriminatory power
x_reduced = selector.fit_transform(x, y)
x_reduced.shape

(1563, 5)

In [8]:
cols = selector.get_support(indices=True)
selected_columns = x.iloc[:,cols].columns.tolist()
selected_columns

['HALSTEAD_EFFORT',
 'HALSTEAD_LENGTH',
 'HALSTEAD_PROG_TIME',
 'HALSTEAD_VOLUME',
 'NUM_OPERATORS']

In [9]:
y.shape

(1563,)

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_reduced, y, test_size=0.3, random_state=10)
print(x.shape)
print(y.shape)
print('-------------------')
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(1563, 37)
(1563,)
-------------------
(1094, 5)
(1094,)
(469, 5)
(469,)


In [11]:
from sklearn.svm import SVC
from sklearn import metrics #for checking the model accuracy
svc = SVC()
svc.fit(x_train, y_train)
svc_pred=svc.predict(x_test)
from sklearn import metrics #for checking the model accuracy
acc_svm = metrics.accuracy_score(svc_pred,y_test)
print('The accuracy of the SVM is:',acc_svm)
print('Th confusion matrix of SVM:\n ',metrics.confusion_matrix(y_test, svc_pred))

The accuracy of the SVM is: 0.908315565031983
Th confusion matrix of SVM:
  [[426   0]
 [ 43   0]]


In [12]:
print(metrics.confusion_matrix(y_test, svc_pred))
print(metrics.classification_report(y_test, svc_pred))

[[426   0]
 [ 43   0]]
              precision    recall  f1-score   support

       False       0.91      1.00      0.95       426
        True       0.00      0.00      0.00        43

    accuracy                           0.91       469
   macro avg       0.45      0.50      0.48       469
weighted avg       0.83      0.91      0.86       469



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2) #put 2 neighbors because we have 2 labels.
knn.fit(x_train,y_train)
knn_pred = knn.predict(x_test)
acc_knn = metrics.accuracy_score(knn_pred,y_test)
print('The accuracy of the KNN is', acc_knn)
print('The confusion matrix of KNN:\n ',metrics.confusion_matrix(y_test, knn_pred))
print(metrics.confusion_matrix(y_test, knn_pred))
print(metrics.classification_report(y_test, knn_pred))

The accuracy of the KNN is 0.8891257995735607
The confusion matrix of KNN:
  [[416  10]
 [ 42   1]]
[[416  10]
 [ 42   1]]
              precision    recall  f1-score   support

       False       0.91      0.98      0.94       426
        True       0.09      0.02      0.04        43

    accuracy                           0.89       469
   macro avg       0.50      0.50      0.49       469
weighted avg       0.83      0.89      0.86       469



In [14]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
model_score = model.score(x_test, y_test)
print(model_score)
print(metrics.confusion_matrix(y_test, y_predict))
print(metrics.classification_report(y_test, y_predict))

0.8507462686567164
[[386  40]
 [ 30  13]]
              precision    recall  f1-score   support

       False       0.93      0.91      0.92       426
        True       0.25      0.30      0.27        43

    accuracy                           0.85       469
   macro avg       0.59      0.60      0.59       469
weighted avg       0.87      0.85      0.86       469



In [15]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x_train, y_train)
model.score(x_train, y_train) 

0.4844606946983547

In [16]:
test_pred = model.predict(x_test)
print(metrics.classification_report(y_test, test_pred))
print(metrics.confusion_matrix(y_test, test_pred))

              precision    recall  f1-score   support

       False       0.97      0.43      0.60       426
        True       0.13      0.86      0.23        43

    accuracy                           0.47       469
   macro avg       0.55      0.65      0.41       469
weighted avg       0.89      0.47      0.56       469

[[183 243]
 [  6  37]]
